<a href="https://colab.research.google.com/github/destello501/destello501.github.io/blob/main/CMSOpenDataZtoMuMu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#panda nos permite manipular el archivo csv https://pandas.pydata.org
import pandas as pd
#numpy nos permite hacer operaciones usando vectores https://numpy.org
import numpy as np
#plotly nos permite crear graficas interactivas https://plotly.com/python/
import plotly.express as px

import plotly.graph_objects as go


%matplotlib inline
#usamos la liga que está directamente en la página.
particles = pd.read_csv('https://opendata.cern.ch/record/5208/files/Zmumu.csv',delimiter=',')
particles.head()


,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,phi2,Q2,dxy2,iso2
0,165617,74969122,54.7055,-0.4324,2.5742,1,-0.0745,0.4999,34.2464,-0.9885,-0.4987,-1,0.0712,3.4221
1,165617,75138253,24.5872,-2.0522,2.8666,-1,-0.0554,0.0000,28.5389,0.3852,-1.9912,1,0.0515,0.0000
2,165617,75887636,31.7386,-2.2595,-1.3323,-1,0.0879,0.0000,30.2344,-0.4684,1.8833,1,-0.0876,0.0000
3,165617,75779415,39.7394,-0.7123,-0.3123,1,0.0585,0.0000,48.2790,-0.1956,2.9703,-1,-0.0492,0.0000
4,165617,75098104,41.2998,-0.1571,-3.0408,1,-0.0305,1.2280,43.4508,0.5910,-0.0428,-1,0.0442,0.0000


In [2]:
fig_5=px.histogram(particles['eta1'],particles['eta1'])
fig_5.show()

In [3]:
fig_5=px.histogram(particles['pt1'],particles['pt1'])
fig_5.show()

In [5]:
# Calcular la masa invariante con la fórmula simplificada
delta_eta = particles['eta1'] - particles['eta2']
delta_phi = particles['phi1'] - particles['phi2']

mass_squared = 2 * particles['pt1'] * particles['pt2'] * (np.cosh(delta_eta) - np.cos(delta_phi))
particles['invariant_mass'] = np.sqrt(mass_squared)

#Graficar con Plotly
fig = px.histogram(particles, x='invariant_mass', nbins=60,
                   title='Masa invariante de dos muones (fórmula simplificada)',
                   labels={'invariant_mass': 'Masa invariante [GeV]'})
fig.update_layout(bargap=0.05)
fig.show()
